# Flow and task execution

## Task runners

- Task runners are responsible for running Prefect tasks within a flow. Each flow has a task runner associated with it. 
- Depending on the task runner you use, the tasks within your flow can run sequentially, concurrently, or in parallel
- You can even configure task runners to use distributed execution infrastructure such as a Dask cluster
- The default task runner is the ConcurrentTaskRunner, which will run submitted tasks concurrently. If you don't specify a task runner, Prefect uses the ConcurrentTaskRunner.

In [1]:
import time
from prefect import task, flow

@task
def my_task():
    return 1

@flow
def my_flow():
    result = my_task.submit()

if __name__ == "__main__":
    my_flow()


22:32:52.972 | INFO    | prefect.engine - Created flow run 'vigilant-bittern' for flow 'my-flow'

22:32:53.066 | INFO    | Flow run 'vigilant-bittern' - Created task run 'my_task-0' for task 'my_task'

22:32:53.067 | INFO    | Flow run 'vigilant-bittern' - Submitted task run 'my_task-0' for execution.

22:32:53.125 | INFO    | Task run 'my_task-0' - Finished in state Completed()

22:32:53.147 | INFO    | Flow run 'vigilant-bittern' - Finished in state Completed('All states completed.')

## Concurrent execution

In [2]:
import time
from prefect import task, flow

@task
def print_values(values):
    for value in values:
        time.sleep(0.5)
        print(value, end="\r")

@flow
def my_flow():
    print_values.submit(["AAAA"] * 15)
    print_values.submit(["BBBB"] * 10)

if __name__ == "__main__":
    my_flow()


/home/dino/anaconda3/envs/torch/lib/python3.10/site-packages/prefect/flows.py:243: UserWarning: A flow named 'my-flow' and defined at '/tmp/ipykernel_23697/3801830279.py:10' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


22:33:59.594 | INFO    | prefect.engine - Created flow run 'sage-bloodhound' for flow 'my-flow'

22:33:59.678 | INFO    | Flow run 'sage-bloodhound' - Created task run 'print_values-0' for task 'print_values'

22:33:59.680 | INFO    | Flow run 'sage-bloodhound' - Submitted task run 'print_values-0' for execution.

22:33:59.691 | INFO    | Flow run 'sage-bloodhound' - Created task run 'print_values-1' for task 'print_values'

22:33:59.692 | INFO    | Flow run 'sage-bloodhound' - Submitted task run 'print_values-1' for execution.

22:34:04.836 | INFO    | Task run 'print_values-1' - Finished in state Completed()

22:34:07.339 | INFO    | Task run 'print_values-0' - Finished in state Completed()

22:34:07.362 | INFO    | Flow run 'sage-bloodhound' - Finished in state Completed('All states completed.')

## Sequential execution

In [3]:
import time
from prefect import task, flow
from prefect.task_runners import SequentialTaskRunner

@task
def print_values(values):
    for value in values:
        time.sleep(0.5)
        print(value, end="\r")

@flow(task_runner=SequentialTaskRunner())
def my_flow():
    print_values.submit(["AAAA"] * 15)
    print_values.submit(["BBBB"] * 10)

if __name__ == "__main__":
    my_flow()


/home/dino/anaconda3/envs/torch/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'print_values' and defined at '/tmp/ipykernel_23697/1318146663.py:5' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/home/dino/anaconda3/envs/torch/lib/python3.10/site-packages/prefect/flows.py:243: UserWarning: A flow named 'my-flow' and defined at '/tmp/ipykernel_23697/1318146663.py:11' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


22:34:47.891 | INFO    | prefect.engine - Created flow run 'arcane-ammonite' for flow 'my-flow'

22:34:47.971 | INFO    | Flow run 'arcane-ammonite' - Created task run 'print_values-0' for task 'print_values'

22:34:47.972 | INFO    | Flow run 'arcane-ammonite' - Executing 'print_values-0' immediately...

22:34:55.541 | INFO    | Task run 'print_values-0' - Finished in state Completed()

22:34:55.556 | INFO    | Flow run 'arcane-ammonite' - Created task run 'print_values-1' for task 'print_values'

22:34:55.557 | INFO    | Flow run 'arcane-ammonite' - Executing 'print_values-1' immediately...

22:35:00.673 | INFO    | Task run 'print_values-1' - Finished in state Completed()

22:35:00.697 | INFO    | Flow run 'arcane-ammonite' - Finished in state Completed('All states completed.')

## Parallel execution

- You can also run tasks using parallel or distributed execution by using the Dask or Ray task runners available through [Prefect Integrations](https://docs.prefect.io/latest/integrations/).

For example, you can achieve parallel task execution, even on in a local execution environment, but using the DaskTaskRunner

- Install the [prefect-dask collection](https://prefecthq.github.io/prefect-dask/) with `pip install prefect-dask`.
- Switch your task runner to the DaskTaskRunner.
- Call .submit on the task instead of calling the task directly. This submits the task to the task runner rather than running the task in-process.

In [4]:
import time
from prefect import task, flow
from prefect_dask.task_runners import DaskTaskRunner

@task
def print_values(values):
    for value in values:
        time.sleep(0.5)
        print(value, end="\r")

@flow(task_runner=DaskTaskRunner())
def my_flow():
    print_values.submit(["AAAA"] * 15)
    print_values.submit(["BBBB"] * 10)

if __name__ == "__main__":
    my_flow()


/home/dino/anaconda3/envs/torch/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'print_values' and defined at '/tmp/ipykernel_23697/3938918850.py:5' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/home/dino/anaconda3/envs/torch/lib/python3.10/site-packages/prefect/flows.py:243: UserWarning: A flow named 'my-flow' and defined at '/tmp/ipykernel_23697/3938918850.py:11' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


22:37:31.210 | INFO    | prefect.engine - Created flow run 'inventive-lynx' for flow 'my-flow'

22:37:31.211 | INFO    | prefect.task_runner.dask - Creating a new Dask cluster with `distributed.deploy.local.LocalCluster`

22:37:31.225 | INFO    | distributed.http.proxy - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy

22:37:31.228 | INFO    | distributed.scheduler - State start

...

22:37:42.053 | INFO    | distributed.core - Removing comms to tcp://127.0.0.1:38179

22:37:42.054 | INFO    | distributed.scheduler - Lost all workers

22:37:42.643 | INFO    | distributed.scheduler - Scheduler closing...

22:37:42.644 | INFO    | distributed.scheduler - Scheduler closing all comms

22:37:42.646 | INFO    | Flow run 'inventive-lynx' - Finished in state Completed('All states completed.')

## Asynchronous execution

In [6]:
import asyncio

from prefect import task, flow

@task
async def print_values(values):
    for value in values:
        await asyncio.sleep(1) # yield
        print(value, end=" ")

@flow
async def async_flow():
    await print_values([1, 2])  # runs immediately
    coros = [print_values("abcd"), print_values("6789")]

    # asynchronously gather the tasks
    await asyncio.gather(*coros)

asyncio.run(async_flow())



/home/dino/anaconda3/envs/torch/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'print_values' and defined at '/tmp/ipykernel_23697/3707094215.py:5' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/home/dino/anaconda3/envs/torch/lib/python3.10/site-packages/prefect/flows.py:243: UserWarning: A flow named 'async-flow' and defined at '/tmp/ipykernel_23697/3707094215.py:11' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


RuntimeError: asyncio.run() cannot be called from a running event loop